In [50]:
!pip install datasets nltk torch googletrans==4.0.0-rc1

In [51]:
# !git clone https://github.com/booydar/babilong source
# !unzip source/data/tasks_1-20_v1-2.zip -d data/

In [52]:
task = 'qa1'
task_path = 'data/tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_test.txt'
message_lengths = [4000]
number_of_samples = 1

In [53]:
from source.babilong_utils import get_dataset_df
import pandas as pd

test_df = get_dataset_df(dataset_path=task_path)

compressed = test_df[test_df.answer.isna()].text.apply(lambda x: " ".join(x.split(" ")[1:]))
print(len(set(list(compressed))))

30


In [54]:
from googletrans import Translator
import os

credentials_path = "/home/bhux/workplace/hamming/translate/translate-dataset-421016-e7b74e6fbd66.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

def translate_text(text, source_lang, target_lang):
    translator = Translator()
    translated_text = translator.translate(text, src=source_lang, dest=target_lang)
    return translated_text.text

In [55]:
import pickle

translation_file = "./translation_test.pkl"

if translation_file:
    with open(translation_file, 'rb') as handle:
        translation = pickle.load(handle)
else:
    translation = {}

    for i in sorted(set(list(compressed))):
        print(i)
        translated_text = translate_text("John " + i, "en", "fr")
        translation[i] = " ".join(translated_text.split(" ")[1:])

    with open('translation_test.pkl', 'wb') as handle:
        pickle.dump(translation, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [56]:
def convert_qa1(x):
    subject = x.split(" ")[0]
    key = " ".join(x.split(" ")[1:])
    translation_text = translation[key]
    return subject + " " + translation_text

test_df.loc[test_df.answer.isna(), "text"] = test_df[test_df.answer.isna()].text.apply(lambda x: convert_qa1(x))

display(test_df)

test_df.to_csv("./fr_qa1_single-supporting-fact_test.txt")

,text,phrase_num,answer,reference_num,initial_sample_num,sample_num
0,John s'est rendu dans le couloir.,1,None,None,0,0
1,Mary s'est rendu aux toilettes.,2,None,None,0,0
2,Where is John?,3,hallway,[1],0,0
3,John s'est rendu dans le couloir.,1,None,None,0,1
4,Mary s'est rendu aux toilettes.,2,None,None,0,1
...,...,...,...,...,...,...
6995,Sandra est retourné au jardin.,10,None,None,199,999
6996,Sandra s'est rendu aux toilettes.,11,None,None,199,999
6997,Sandra a déménagé au bureau.,13,None,None,199,999
6998,Daniel s'est rendu au jardin.,14,None,None,199,999


In [57]:
# from googletrans import Translator
# credentials_path = "/home/bhux/workplace/hamming/translate/translate-dataset-421016-e7b74e6fbd66.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
# translated_text = translate_text("hello", "en", "fr")
# print(translated_text)